#Lista 2
-------------


**Objetivo**: Treinar modelos de classificação utilizando Random Forest e XGBoost.

**Banco de Dados**: Detecção de Doenças no Fígado


[Link](https://drive.google.com/file/d/1jnLwuv4e_ZeCqluMXs3Mar2TGetumtVv/view?usp=drive_link) para o banco


Descrição do banco:
> Este banco de dados contém registros de 416 pacientes diagnosticados com doença no fígado e 167 pacientes sem a doença. Esta informação está na coluna `Selector`.
>
> Há 10 variáveis no banco:
>
> * age: idade
> * Gender: gênero do paciente
> * TB: Bilirrubina total
> * DB: Bilirrubina direta
> * Alkphos: fosfatase alcalina.
> * Sgpt: transaminase glutâmico-pirúvica sérica (TGP)
> * Sgot: transaminase glutâmico-oxalacética sérica (TGO)
> * TP: Proteína total
> * ALB: Albumina
> * A/G Ratio: Relação Albumina:Globulina
> * Selector: 1 - No | 2 - Yes


**Exercício**:
1. Carregue o banco de dados e analise suas features - [OK]

* Transforme a feature Gender em uma variável dummy - [0K]

2. Separe o banco de dados:

* Utilize 80% dos dados para treino e 20% para teste - [OK]
3. Treine um modelo de Random Forest:

* Utilize os parâmetros n_estimators=150 e max_depth=5.- [OK]
* Pergunta: Quais são as duas features mais importantes?

4. Treine um modelo de XGBoost:

* Utilize os parâmetros max_depth=5, learning_rate=0.1 e n_estimators=150.
* Pergunta: Quais são as duas features mais importantes?

5. Construa um relatório comparativo dos dois modelos:

* Utilizando a **precisão da classe de pacientes doentes**, qual modelo tem melhor desempenho na detecção de doença no fígado?
* Utilizando o **f1-score da classe de pacientes doentes**, qual modelo tem melhor desempenho na detecção de doença no fígado?


Obs.: Utilize o **mesmo** conjunto de treino e teste para construir e avaliar ambos os modelos.



## 1. Carga do banco
--------------------

In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('./dataset/indian_liver.csv')

### 1.1 Formatação e preenchimento de values nulos

In [2]:
# Transform gender into dummy variablesî
df = pd.get_dummies(df, columns=['Gender'], drop_first=False)


In [3]:

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('/', '_')
df['a_g_ratio'] = df['a_g_ratio'].fillna(df['a_g_ratio'].median())

### 1.2 Analise dos Dados

In [4]:
df.describe()

,age,tb,db,alkphos,sgpt,sgot,tp,alb,a_g_ratio,selector
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.946947,1.286449
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.318495,0.452490
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000,1.000000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000,1.000000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.930000,1.000000
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000,2.000000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000,2.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            583 non-null    int64  
 1   tb             583 non-null    float64
 2   db             583 non-null    float64
 3   alkphos        583 non-null    int64  
 4   sgpt           583 non-null    int64  
 5   sgot           583 non-null    int64  
 6   tp             583 non-null    float64
 7   alb            583 non-null    float64
 8   a_g_ratio      583 non-null    float64
 9   selector       583 non-null    int64  
 10  gender_female  583 non-null    bool   
 11  gender_male    583 non-null    bool   
dtypes: bool(2), float64(5), int64(5)
memory usage: 46.8 KB


In [6]:
fig = px.violin(df, y='age', x='selector', 
                    box=True, 
                    points="all",
                    title='Age distribution by selector', 
                    labels={'selector': 'Liver disease'}, 
                    animation_group='selector',
                    category_orders={'selector': ['1', '2']}, color='selector')
fig.show()

In [7]:
fig = px.scatter(df, x='sgpt', y='sgot', color='sgpt', title='TGO and TGP levels')
fig.show()

In [8]:
# Plot a histogram of the age distribution
fig = px.histogram(df, x='alkphos', 
                       title='Aalkphos', 
                       labels={'alkphos': 'Alkphos', 'count': 'Number of patients'})

fig.show()

In [9]:
fig = px.scatter(df, x='age', y='alkphos', color='selector', title='Total Bilirubin vs Age', labels={'age': 'Age', 'total_bilirubin': 'Total Bilirubin', 'selector': 'Liver Disease'})
fig.show()

## 2. Quebra do banco
--------------------

In [10]:
from sklearn.model_selection import train_test_split

Y = df['selector']
X = df.drop(columns=['selector'])


# 80% training and 20% test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## 3. Treino do Primeiro modelo - Random Forest
--------------------

In [11]:
# Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=150, max_depth=5, random_state=42)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

print(Y_pred)

[1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 2
 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 1 2 1 1 1 2]


In [12]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           1       0.76      0.89      0.82        87
           2       0.38      0.20      0.26        30

    accuracy                           0.71       117
   macro avg       0.57      0.54      0.54       117
weighted avg       0.66      0.71      0.68       117



In [13]:
confusion_matrix(Y_test, Y_pred)

array([[77, 10],
       [24,  6]])

### 3.1 Interpretação de dois coeficientes
--------------------

#### Random Forest - Resultado

- **Desempenho na Classe 1**: O modelo tem bom desempenho ao classificar exemplos da Classe 1, com alta precisão e recall. Isso significa que ele consegue distinguir bem essa classe.

- **Desempenho na Classe 2**: O modelo apresenta um desempenho fraco na Classe 2, com baixa precisão e revocação, indicando que ele tem dificuldade para identificar corretamente os exemplos dessa classe. Isso pode ser resultado de um desequilíbrio de classes ou uma incapacidade do modelo de capturar as características da Classe 2.

- **Desequilíbrio de Classe**: A Classe 1 tem um número significativamente maior de exemplos (87 contra 30), o que pode estar levando o modelo a um viés em favor da Classe 1. Essa diferença de suporte é refletida nas métricas globais ponderadas.

##### Resumo

O modelo tem um bom desempenho geral (acurácia de 71%) e se sai bem na Classe 1, mas apresenta dificuldades significativas na Classe 2. Ajustes e técnicas de balanceamento podem ajudar a aumentar a precisão e a revocação da Classe 2, o que pode ser crucial dependendo da importância dessa classe no problema real.


## 4. Segundo modelo - XGBoost
--------------------

1. **binary:logistic**
Descrição: Para problemas de classificação binária. Retorna a probabilidade de uma classe (geralmente 1).
Uso: Quando você está lidando com um problema de classificação binária e deseja obter probabilidades como saída.

2. **binary:logitraw**
Descrição: Semelhante ao binary:logistic, mas retorna o valor logit (antes da aplicação da função sigmoide) em vez da probabilidade.
Uso: Útil se você deseja aplicar uma transformação adicional nas previsões.

3. **multi:softmax**
Descrição: Para problemas de classificação multiclasse. Retorna a classe com a maior probabilidade.
Uso: Quando você precisa de uma previsão de classe discreta (sem probabilidades).

4. **multi:softprob**
Descrição: Semelhante ao multi:softmax, mas retorna as probabilidades para cada classe.
Uso: Útil quando você precisa de informações sobre a confiança da previsão em relação a cada classe.

5. **rank:pairwise**
Descrição: Usado para problemas de ranking. O modelo aprende a classificar pares de amostras.
Uso: Aplicável em cenários como recomendação de produtos e mecanismos de busca.

6. **rank:ndcg**
Descrição: Um método de ranking que maximiza a métrica NDCG (Normalized Discounted Cumulative Gain).
Uso: Para cenários onde a qualidade do ranking é mais importante do que a precisão.

7. **rank:map**
Descrição: Um método de ranking que maximiza a média da precisão (Mean Average Precision).
Uso: Similar ao rank:ndcg, mas focado em maximizar a precisão.

8. **binary:hinge**
Descrição: Usado para classificação binária com a função de perda hinge, que é comum em máquinas de vetores de suporte (SVM).
Uso: Quando se deseja um modelo robusto em relação a outliers.

In [24]:
import xgboost as xgb

# Utilize os parâmetros max_depth=5, learning_rate=0.1 e n_estimators=150.
XG_model = xgb.XGBClassifier(n_estimators=150, max_depth=5, learning_rate=0.1)


In [21]:
# Normalize the Train and Test sets to be used in the XGBoost model
# The XGBoost model requires the target variable to start from 0
Y_train_adjusted = Y_train - 1
Y_test_adjusted = Y_test - 1

In [28]:
# Fit the model
XG_model.fit(X_train, Y_train_adjusted)

# Predict the model
XG_predict = XG_model.predict(X_test)

In [33]:
print(classification_report(Y_test_adjusted, XG_predict))

              precision    recall  f1-score   support

           0       0.78      0.83      0.80        87
           1       0.40      0.33      0.36        30

    accuracy                           0.70       117
   macro avg       0.59      0.58      0.58       117
weighted avg       0.68      0.70      0.69       117



In [29]:
XG_predict

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1])

#### Resultado XGBoost

##### Classe 0
- **Precisão:** A Precisão do modelo foi de 78%, indicando que de 100 tentativas foram feitos 78 acertos na classe 0.

- **Recall:** O Recall foi de 83% o que significa que o modelo consegue identificar em 100 ocorrencias 83 de forma assertiva.

- **F1-Score:** O F1-Score foi de 80%, o que representa a média harmonica entre a previsão e o recall, demonstrando um bom equilibrio para a Classe 0.

##### Classe 1
- **Precisão:** A Precisão do modelo foi de 40%, indicando que de 100 tentativas foram feitos 40 acertos na classe 1.

- **Recall:** O Recall foi de 33% o que significa que o modelo consegue identificar em 100 ocorrencias 33 de forma assertiva.

- **F1-Score:** O F1-Score foi de 36%, o que representa a média harmonica entre a previsão e o recall, demonstrando um bom equilibrio para a Classe 1.

##### Analise Final
O Modelo apresenta um desequilibrio na predição dos resultados quando compramos a Classe 0 e 1. Onde existe um desempenho muito bom Acima de 70% na Classe 1 e um desempenho muito baixo na classe 1.
- Desta forma, entendemos que a distribuição dos dados pode estar afetando os resultados onde existe um desequilibrio entre as 2 classes, tornando o resultado da classe 1 pior. 




## 4.1 Duas features mais importantes
--------------------

In [43]:
importances = XG_model.feature_importances_

features = X.columns

df_importances = pd.DataFrame(importances, index=features, columns=['importance'])

df_importances = df_importances.sort_values(by='importance', ascending=False)

fig = px.bar(df_importances, y='importance', x=df_importances.index, title='Feature importances', labels={'index': 'Feature', 'importance': 'Importance'})

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)

fig.add_annotation(x='db', y=0.1, text='Primeira Maior Relevancia', showarrow=True, yshift=10)
fig.add_annotation(x='tb', y=0.1, text='Segunda Maior Relevancia', showarrow=True, yshift=10)


fig.show()


## 5. Avaliação do modelo
--------------------

#### 5.1 - Resultado do Modelo

O modelo XGBoost tem um bom desempenho na classe 1 (com boas métricas de precisão, recall e F1-score), mas apresenta problemas na classe 2, com precisão e recall baixos. Isso pode ser causado pelo desequilíbrio no número de exemplos entre as duas classes, que afeta o desempenho na classe com menos dados (classe 2). Dependendo do objetivo, pode ser necessário ajustar o modelo para melhorar o desempenho na classe 2, como usar técnicas de balanceamento de classes ou ajustar hiperparâmetros.

### 5.1 Qual modelo escolhido olhando para a precisão de detecção de doença no fígado?
--------------------

R: Com base nos resultados, o Random Forest seria a escolha, apresenta um desempenho um pouco melhor em em relação a classe minoritária, o que podemos considerar como importante para evitar falsos negativos em diagnosticos,  o RF também e oferece boa precisão na classe majoritária.

### 5.2 Qual modelo escolhido olhando para o f1-score do rótulo de doença do fígado?
--------------------

R: XGBoost apresenta uma escolha mais adequada, pois melhora ligeiramente o equilíbrio entre precisão e revocação nas duas classes. Onde podemos comparar

| Métrica          | Random Forest | XGBoost |
|------------------|---------------|---------|
| **Classe Majoritária (1 / 0)** | **0.82**        | 0.80    |
| **Classe Minoritária (2 / 1)** | 0.26           | **0.36** |
| **Macro F1-Score** | 0.54           | **0.58** |
| **Weighted F1-Score** | 0.68           | **0.69** |

